In [11]:
import awkward as ak
import h5py

ModuleNotFoundError: No module named 'awkward'

In [3]:
# load the data from the skimmed file

file='skimmed_ttbarDM.h5'

with h5py.File(file, 'r') as f:
    # Load the form and length metadata
    form = f.attrs['form']
    length = f.attrs['length']
    
    # Load the buffers (numpy arrays) from the HDF5 file
    container = {key: f[key][()] for key in f.keys()}
    
    # Reconstruct the Awkward Array from buffers
    df = ak.from_buffers(form, length, container)
print(ak.__version__)

AttributeError: module 'awkward' has no attribute 'from_buffers'

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from pathlib import Path as Path
import cpt_columnar
from functools import partial
from tqdm import tqdm
import wandb
tqdm = partial(tqdm, position=0, leave=True)
from cpt_columnar import ColumnarCovariantTopFormer
import matplotlib.pylab as pylab
%matplotlib inline
%load_ext autoreload
%autoreload 2

USE_GPU = True
dtype = torch.float32 # we will be using float
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

from seaborn import heatmap
import seaborn as sns
sns.set_style("white")
sns.axes_style("white")

using device: cpu



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/bachjoer/anaconda3/envs/cpt/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/bachjoer/anaconda3/envs/cpt/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/bachjoer/anaconda3/envs/cpt/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/bachjoer/anaconda3/envs/cpt/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in l

{'figure.facecolor': 'white',
 'axes.labelcolor': '.15',
 'xtick.direction': 'out',
 'ytick.direction': 'out',
 'xtick.color': '.15',
 'ytick.color': '.15',
 'axes.axisbelow': True,
 'grid.linestyle': '-',
 'text.color': '.15',
 'font.family': ['sans-serif'],
 'font.sans-serif': ['Arial',
  'DejaVu Sans',
  'Liberation Sans',
  'Bitstream Vera Sans',
  'sans-serif'],
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'image.cmap': 'rocket',
 'xtick.top': False,
 'ytick.right': False,
 'axes.grid': False,
 'axes.facecolor': 'white',
 'axes.edgecolor': '.15',
 'grid.color': '.8',
 'axes.spines.left': True,
 'axes.spines.bottom': True,
 'axes.spines.right': True,
 'axes.spines.top': True,
 'xtick.bottom': False,
 'ytick.left': False}

In [2]:
max_num_output = 2 # assert that there are at most ${max_num_output} tops
base_dir = './trained'
arch = 'ColumnarCovariantTopFormer'
model_params = {
    'in_dim': 9,
    'out_dim': 4,
    'max_num_output': max_num_output,
    'hidden_dim': 256,
    'num_convs': (6, 6),
    'heads': 4,
    'mass': 173,
    'match_scale_factor': torch.FloatTensor([0, 1, 1, 0]), # used for matching dR = (dy, dphi)    
    'p_norm': 2, # used in matching and loss
    'beta': 0.8, # loss weight for predicting number of tops
    'dropout': 0.,
    'schedule_lr': False,
    'use_gpu': USE_GPU,
    'uniform_attention': False,
}
tag = 'TEST' # CHANGE THIS
output_dir = f"{arch}_{model_params['num_convs']}_{model_params['hidden_dim']}_{tag}"
output_dir = os.path.join(base_dir, output_dir)
model_params['output_dir'] = output_dir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model = eval(arch)(**model_params).to(device)

Using Lamb optimizer with lr=0.0001
Initializing a new model from scratch
Model has 9.9M parameters


In [14]:

from torch.utils.data import Dataset, DataLoader

class RandomFloatDataset(Dataset):
    def __init__(self, num_samples, num_features=9):
        self.num_samples = num_samples
        self.num_features = num_features

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Return a random float tensor of size (num_features,)
        return torch.randn(self.num_features)

# Number of samples in the dataset
num_samples = 1000

# Create the dataset
dataset = RandomFloatDataset(num_samples)

# Create the DataLoader
batch_size = 256
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

# Example of iterating over the DataLoader
for batch in dataloader:
    print(batch)  # Each batch is a tensor of size (batch_size, num_features)

    break  # Remove this break to see more batches


tensor([[ 0.8705, -0.3191, -1.2725,  ...,  0.7527, -0.1338,  0.8917],
        [-0.8603, -0.2934, -0.0837,  ..., -0.0994, -0.6235,  1.3766],
        [-0.8109,  0.4782, -0.7009,  ..., -1.5940, -1.2899, -0.5408],
        ...,
        [ 0.5878,  2.0120,  0.6503,  ...,  1.0011, -0.1303, -1.8872],
        [ 0.9711, -0.0268,  0.9017,  ...,  0.3363,  0.7519, -0.1950],
        [-1.4853,  0.8614,  0.3373,  ..., -0.8464,  0.5913,  0.1306]])


In [15]:
for batch in dataloader:
    out = model(batch) 
    print(out)
  

torch.Size([1, 256, 512])
torch.Size([1, 256, 512])
psource is torch.Size([256, 3])
torch.Size([256])
(tensor([[[ 2.0364e+01, -2.8822e-01, -9.9933e-01,  3.6617e-02,  1.6764e+02],
         [ 2.0364e+01, -2.8822e-01, -9.9933e-01,  3.6617e-02,  1.6764e+02]],

        [[-3.0296e+00, -1.2431e+00,  1.2160e-01,  9.9258e-01,  1.7450e+02],
         [-3.0296e+00, -1.2431e+00,  1.2160e-01,  9.9258e-01,  1.7450e+02]],

        [[-1.7079e+01, -4.4250e-01, -9.8026e-01,  1.9772e-01,  1.7027e+02],
         [-1.7079e+01, -4.4250e-01, -9.8026e-01,  1.9772e-01,  1.7027e+02]],

        ...,

        [[ 4.8523e+01, -9.6860e-01,  9.0492e-01, -4.2558e-01,  1.7093e+02],
         [ 4.8523e+01, -9.6860e-01,  9.0492e-01, -4.2558e-01,  1.7093e+02]],

        [[ 4.2557e+01, -2.3698e+00, -7.3611e-01, -6.7686e-01,  1.7308e+02],
         [ 4.2557e+01, -2.3698e+00, -7.3611e-01, -6.7686e-01,  1.7308e+02]],

        [[-2.8746e+01,  1.6250e-04, -6.8221e-01,  7.3115e-01,  1.7068e+02],
         [-2.8746e+01,  1.6250e-04, -

/home/bachjoer/workspace/phd/ml-workspace/Covariant-Particle-Transformer/cpt_columnar.py:555: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(e) # attend over input


(tensor([[[-2.9120e+00, -5.6389e-01, -9.9127e-01, -1.3185e-01,  1.7022e+02],
         [-2.9120e+00, -5.6389e-01, -9.9127e-01, -1.3185e-01,  1.7022e+02]],

        [[-8.6938e+00,  1.2162e-01,  8.3639e-01, -5.4813e-01,  1.7172e+02],
         [-8.6938e+00,  1.2162e-01,  8.3639e-01, -5.4813e-01,  1.7172e+02]],

        [[ 8.1430e+01, -4.7247e-01, -9.9970e-01, -2.4459e-02,  1.7136e+02],
         [ 8.1430e+01, -4.7247e-01, -9.9970e-01, -2.4459e-02,  1.7136e+02]],

        ...,

        [[-3.6755e+01, -1.2841e+00, -9.6497e-01,  2.6236e-01,  1.7426e+02],
         [-3.6755e+01, -1.2841e+00, -9.6497e-01,  2.6236e-01,  1.7426e+02]],

        [[ 4.0305e+01, -7.7625e-01,  9.4500e-01, -3.2708e-01,  1.7421e+02],
         [ 4.0305e+01, -7.7625e-01,  9.4500e-01, -3.2708e-01,  1.7421e+02]],

        [[ 1.9392e+01, -1.3580e+00, -9.8487e-01,  1.7329e-01,  1.7385e+02],
         [ 1.9392e+01, -1.3580e+00, -9.8487e-01,  1.7329e-01,  1.7385e+02]]],
       grad_fn=<CatBackward0>), tensor([-0.1789,  0.5978], gr